In [ ]:
pip install tensorflow

In [ ]:
pip install tensorflow_recommenders

In [96]:
import pymysql
import pandas as pd

host = '34.64.95.29'
port = 3306
user = 'root'
password = 'acorn1234'
database = 'music'

connection = pymysql.connect(host=host, port=port, user=user, password=password, database=database)

new_song = pd.read_sql_query('SELECT * FROM new_song', connection)

connection.close()


C:\Users\401-14\AppData\Local\Temp\ipykernel_28180\841987660.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  new_song = pd.read_sql_query('SELECT * FROM new_song', connection)


In [106]:
import numpy as np

# 기존 데이터의 평균과 표준편차
mean_likes = 500
std_likes = 350

# new_song 데이터에 like 열 추가
new_song['like'] = np.random.normal(loc=mean_likes, scale=std_likes, size=len(new_song))

# 좋아요 숫자는 음수가 되지 않도록 조정
new_song['like'] = new_song['like'].apply(lambda x: max(0, x))
new_song['like'] = new_song['like'].apply(lambda x:int(x))

In [107]:
new_song

,id,title,singer,lyric_str,sentiment_score,age,gender,like
0,0,떠나보낼 준비해 둘걸 그랬어,임한별,너랑 입으려 고른 옷 오늘 받아봤어 오래 걸려서 잊고 있었는데 곧 있을 너의 생일에...,0,2,0,476
1,1,매일 멀어지는 사이,신용재 (2F),우리의 시간은 여기 멈춰버렸고 굳어버린 채로 멍하니 서서 또 오늘에서 매일 멀어져 ...,3,2,0,859
2,2,U R,태연 (TAEYEON),오랫동안 내 마음엔 구름 가득 비가 내려 따스한 햇살 비추길 간절히 바랬죠 어깨를 ...,3,2,1,1172
3,3,비로소 너에게 도착했다 (Full Bloom),마크툽 (MAKTUB),너 햇살처럼 다가와 나의 그늘에 아늑히 내려앉아 긴 하루와 지친 맘을 안아주면 나 ...,1,2,0,0
4,4,바라보기,엠씨더맥스 (M.C the MAX),마음이 알고 가 낯선 길인데도 니 숨결이 머문 자리는 너무 그립다 그립다 못해 아프...,2,3,0,593
...,...,...,...,...,...,...,...,...
9233,9233,3500 (Feat. Future & 2 Chainz),Travis Scott,bandana wrappedwrapped round head bandana wrap...,3,2,0,713
9234,9234,Mamacita (Feat. Rich Homie Quan & Young Thug),Travis Scott,mamacita cita cita mamacita cita cita last day...,3,2,0,0
9235,9235,좋은 하루,전상근,어젠 비가 그쳤지 오랜만에 보인 별빛 그걸로 충분하던 날 별 것 아닌 일이지만 낮엔...,2,2,0,0
9236,9236,Cursive,Billie Marten,write cursive lines need help miss shoulder kn...,3,1,1,940


In [303]:
import pandas as pd
import numpy as np
from annoy import AnnoyIndex
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF 벡터화
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(new_song['lyric_str'])
tfidf_matrix_dense = tfidf_matrix.toarray()

annoy_index = AnnoyIndex(tfidf_matrix_dense.shape[1], 'angular')  # cosine 유사도 사용
for i in range(tfidf_matrix_dense.shape[0]):
    annoy_index.add_item(i, tfidf_matrix_dense[i])
annoy_index.build(100)  # 100개의 트리를 사용하여 인덱스를 구축




True

In [316]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(new_song['lyric_str'])
tfidf_matrix_dense = tfidf_matrix.toarray()

In [315]:
annoy_index = AnnoyIndex(tfidf_matrix_dense.shape[1], 'angular')  # cosine 유사도 사용
for i in range(tfidf_matrix_dense.shape[0]):
    annoy_index.add_item(i, tfidf_matrix_dense[i])
annoy_index.build(100)  # 100개의 트리를 사용하여 인덱스를 구축

True

In [319]:
import pickle

# # annoy_index를 피클로 저장
# with open('C:/Users/401-14/Desktop/30-200/annoy_index.pkl', 'wb') as f:
#     pickle.dump(annoy_index, f)

# tfidf_matrix_dense를 피클로 저장
with open('C:/Users/401-14/Desktop/30-200/tfidf_matrix_dense.pkl', 'wb') as f:
    pickle.dump(tfidf_matrix_dense, f)


In [318]:
# annoy_index를 파일로 저장
annoy_index.save('C:/Users/401-14/Desktop/30-200/annoy_index.ann')

True

In [286]:
def recommend_songs(song_ids, annoy_index, tfidf_matrix, num_neighbors=6):
    # song_ids가 int형이면 리스트로 변환
    if isinstance(song_ids, int):
        song_ids = [song_ids]

    recommended_songs = []
    
    if len(song_ids)==1:
        query_vector = tfidf_matrix[song_ids[0]]
        neighbors = annoy_index.get_nns_by_vector(query_vector, num_neighbors)
        recommended_songs.append(neighbors[1:6])
        return recommended_songs[0]
    
    if len(song_ids)==2:
        query_vector = tfidf_matrix[song_ids[0]]
        neighbors = annoy_index.get_nns_by_vector(query_vector, 4)
        recommended_songs.append(neighbors[1])
        recommended_songs.append(neighbors[2])
        recommended_songs.append(neighbors[3])

        query_vector = tfidf_matrix[song_ids[1]]
        neighbors = annoy_index.get_nns_by_vector(query_vector, 3)
        recommended_songs.append(neighbors[1])
        recommended_songs.append(neighbors[2])
        return recommended_songs
    if len(song_ids)==3:
        query_vector = tfidf_matrix[song_ids[0]]
        neighbors = annoy_index.get_nns_by_vector(query_vector, 3)
        recommended_songs.append(neighbors[1])
        recommended_songs.append(neighbors[2])
        query_vector = tfidf_matrix[song_ids[1]]
        neighbors = annoy_index.get_nns_by_vector(query_vector, 3)
        recommended_songs.append(neighbors[1])
        recommended_songs.append(neighbors[2])
        query_vector = tfidf_matrix[song_ids[2]]
        neighbors = annoy_index.get_nns_by_vector(query_vector, 2)
        recommended_songs.append(neighbors[1])
        return recommended_songs
    if len(song_ids)==4:
        query_vector = tfidf_matrix[song_ids[0]]
        neighbors = annoy_index.get_nns_by_vector(query_vector, 3)
        recommended_songs.append(neighbors[1])
        recommended_songs.append(neighbors[2])
        query_vector = tfidf_matrix[song_ids[1]]
        neighbors = annoy_index.get_nns_by_vector(query_vector, 2)
        recommended_songs.append(neighbors[1])
        query_vector = tfidf_matrix[song_ids[2]]
        neighbors = annoy_index.get_nns_by_vector(query_vector, 2)
        recommended_songs.append(neighbors[1])
        query_vector = tfidf_matrix[song_ids[3]]
        neighbors = annoy_index.get_nns_by_vector(query_vector, 2)
        recommended_songs.append(neighbors[1])
        return recommended_songs
    if len(song_ids)==5:
        query_vector = tfidf_matrix[song_ids[0]]
        neighbors = annoy_index.get_nns_by_vector(query_vector, 2)
        recommended_songs.append(neighbors[1])
        query_vector = tfidf_matrix[song_ids[1]]
        neighbors = annoy_index.get_nns_by_vector(query_vector, 2)
        recommended_songs.append(neighbors[1])
        query_vector = tfidf_matrix[song_ids[2]]
        neighbors = annoy_index.get_nns_by_vector(query_vector, 2)
        recommended_songs.append(neighbors[1])
        query_vector = tfidf_matrix[song_ids[3]]
        neighbors = annoy_index.get_nns_by_vector(query_vector, 2)
        recommended_songs.append(neighbors[1])
        query_vector = tfidf_matrix[song_ids[4]]
        neighbors = annoy_index.get_nns_by_vector(query_vector, 2)
        recommended_songs.append(neighbors[1])
        return recommended_songs    


In [313]:
recommend_songs((1,2,3), annoy_index, tfidf_matrix_dense)

[357, 7936, 8490, 1717, 2773]